---

## 📋 Prerequisites

Before running this notebook, ensure you have:

### Azure Resources
- ✅ Azure AI Foundry project
- ✅ Azure OpenAI endpoint with model deployment (e.g., gpt-4o-mini)
- ✅ Azure Container Registry
- ✅ Proper RBAC permissions

### Developer Tools
- ✅ Azure CLI (`az` command)
- ✅ Azure Developer CLI (`azd` command)
- ✅ Docker Desktop (for containerization)
- ✅ Python 3.10+

### Python Packages
```bash
# Core MAF V2 packages (from local or published source)
pip install azure-ai-projects>=2.0.0b1  # Azure AI Projects SDK V2
pip install agent-framework  # Microsoft Agent Framework
pip install agent-framework-azure  # Azure-specific components

# Hosting adapter for Azure AI Foundry
pip install azure-ai-agentserver-core
pip install azure-ai-agentserver-agentframework

# Additional dependencies
pip install azure-identity
pip install python-dotenv
```

### Environment Variables (.env file)
```bash
AZURE_AI_PROJECT_ENDPOINT=https://your-project.services.ai.azure.com/api/projects/your-project
AZURE_AI_MODEL_DEPLOYMENT_NAME=gpt-4o-mini
ACR_NAME=yourregistry
FOUNDRY_ACCOUNT_NAME=your-foundry-account
```

---

## 1️⃣ DEVELOP with MAF

Let's create a simple customer support agent using the Microsoft Agent Framework.

In [1]:
# Import required libraries for MAF V2
from agent_framework.azure import AzureAIClient
from azure.identity import DefaultAzureCredential, AzureCliCredential
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# V2 uses AzureAIClient instead of separate ChatClient
print("✅ MAF V2 imports successful!")
print(f"📁 Project Endpoint: {os.getenv('AZURE_AI_PROJECT_ENDPOINT', 'Not set')}")
print(f"🤖 Model Deployment: {os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME', 'Not set')}")

✅ MAF V2 imports successful!
📁 Project Endpoint: https://r2d2-foundry-001.services.ai.azure.com/api/projects/Main-Project
🤖 Model Deployment: gpt-4.1


In [2]:
# Create a simple customer support agent using MAF V2 AzureAIClient
# The AzureAIClient automatically reads AZURE_AI_PROJECT_ENDPOINT and 
# AZURE_AI_MODEL_DEPLOYMENT_NAME from environment variables

# IMPORTANT: V2 requires using async context managers
# We'll create the agent in an async function for proper resource management

print("Creating agent using MAF V2 pattern...")
print(f"📁 Project Endpoint: {os.getenv('AZURE_AI_PROJECT_ENDPOINT', 'Not set')}")
print(f"🤖 Model Deployment: {os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME', 'Not set')}")

# Note: We'll create the agent inside an async context manager in the next cell
# This ensures proper resource cleanup and follows the official V2 pattern

Creating agent using MAF V2 pattern...
📁 Project Endpoint: https://r2d2-foundry-001.services.ai.azure.com/api/projects/Main-Project
🤖 Model Deployment: gpt-4.1


In [3]:
# Test the agent locally using MAF V2
import asyncio
from azure.identity.aio import AzureCliCredential

test_messages = [
    "What warranty comes with your laptops?",
    "Do you offer free shipping?",
    "I need help tracking my order #12345"
]

print("🧪 Testing agent locally...\n")

async def test_agent():
    # V2 CORRECT PATTERN: Use async context managers!
    # AzureAIClient automatically reads environment variables:
    # - AZURE_AI_PROJECT_ENDPOINT
    # - AZURE_AI_MODEL_DEPLOYMENT_NAME
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(credential=credential).create_agent(
            name="ContosoSupportAgent",
            instructions="""
            You are a helpful customer support agent for Contoso Electronics.
            
            Your responsibilities:
            - Answer product questions about laptops, tablets, and accessories
            - Help customers track orders
            - Provide troubleshooting guidance
            - Be friendly, professional, and concise
            
            Product Information:
            - All laptops come with a 1-year warranty
            - Free shipping on orders over $500
            - 30-day return policy on all products
            """,
            # Optional: Reuse existing agent version
            # use_latest_version=True
        ) as agent,
    ):
        print(f"✅ Agent created: {agent.name} (ID: {agent.id})\n")
        
        # Test each message
        for message_text in test_messages:
            print(f"👤 User: {message_text}")
            
            # Run the agent with a string query (V2 pattern)
            result = await agent.run(message_text)
            
            # Convert result to string
            reply = str(result)
            
            print(f"🤖 Agent: {reply}\n")
            print("-" * 80 + "\n")

# Run the async test
await test_agent()

🧪 Testing agent locally...

✅ Agent created: ContosoSupportAgent (ID: 8e65bcb1-da51-467b-a0c2-2d66281165d0)

👤 User: What warranty comes with your laptops?
🤖 Agent: All of our laptops come with a 1-year warranty. If you have any issues within the first year, we're here to help with repairs or replacements covered by the warranty. Let me know if you have more questions or need details about a specific model!

--------------------------------------------------------------------------------

👤 User: Do you offer free shipping?
🤖 Agent: Yes, we offer free shipping on all orders over $500. If you have any other questions about shipping or need help with an order, just let me know!

--------------------------------------------------------------------------------

👤 User: I need help tracking my order #12345
🤖 Agent: I’d be happy to help you track your order #12345!

Could you please provide the name or email address used to place the order? This will help me locate your order status quickly.

### ✅ Stage 1 Complete!

We've successfully:
- ✅ Created an agent using MAF SDK
- ✅ Defined custom instructions
- ✅ Tested locally with `agent.run()`

**Key Takeaway**: This is standard MAF development - no hosting concerns yet!

---

## 2️⃣ ADAPT for Hosting with Hosting Adapter

The **hosting adapter** is a framework abstraction layer that automatically transforms your MAF agent into an Azure AI Foundry-compatible HTTP service.

### Key Benefits:
- 🚀 **One-line deployment**: `from_agent(my_agent).run()` starts server on `localhost:8088`
- 🔄 **Auto protocol translation**: Converts between Foundry API and MAF formats
- ✅ **Built-in features**: OpenTelemetry, CORS, SSE streaming, structured logging
- 🏭 **Enterprise-ready**: Automatic conversation management, auth flows

### Official Packages
```bash
pip install azure-ai-agentserver-core
pip install azure-ai-agentserver-agentframework
```

### 📦 Package Availability Note

**Important**: The Microsoft Agent Framework and hosting adapter packages are currently in preview and may not be publicly available on PyPI yet.

**What you can do now:**
- ✅ **Stages 1-3**: Development, local testing, and containerization work with the local MAF installation
- ⚠️ **Stage 4-5**: Hosted deployment packages may need to be installed from private feeds or await public release

**For production hosted agents:**
- Check the [Azure AI Foundry documentation](https://learn.microsoft.com/azure/ai-foundry/agents) for the latest package availability
- Consider deploying to Azure Container Apps or AKS as an alternative to Foundry Hosted Agents
- Monitor the [MAF GitHub repository](https://github.com/microsoft/agent-framework) for release announcements

### Create Hosted Agent Server

Using the hosting adapter, we can create a web server that exposes our agent through the Azure AI Foundry Responses API.

In [8]:
# Create agent and hosting adapter server
import asyncio
from azure.identity.aio import AzureCliCredential
from azure.ai.agentserver.agentframework import from_agent_framework
from agent_framework.azure import AzureAIClient

print("Creating hosted agent server...")
print("")
print("Note: In a notebook, we create the agent and server object for demonstration.")
print("In production (app.py), the agent and server persist for the application lifetime.")
print("")

async def setup_hosted_agent():
    """Create agent and hosting adapter server"""
    # Create agent using async context manager
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(credential=credential).create_agent(
            name="ContosoSupportAgent",
            instructions="""
            You are a helpful customer support agent for Contoso Electronics.
            
            Your responsibilities:
            - Answer product questions about laptops, tablets, and accessories
            - Help customers track orders
            - Provide troubleshooting guidance
            - Be friendly, professional, and concise
            
            Product Information:
            - All laptops come with a 1-year warranty
            - Free shipping on orders over $500
            - 30-day return policy on all products
            """,
        ) as agent,
    ):
        print(f"✅ Agent created: {agent.name} (ID: {agent.id})")
        print("")
        
        # Create hosting adapter server
        # This wraps the agent with REST API endpoints
        server = from_agent_framework(agent)
        
        print("✅ Server object created")
        print("   Provides endpoints: POST /responses, GET /health")
        print("")
        print("ℹ️ To run locally: server.run(host='0.0.0.0', port=8088)")
        print("   In production: Use app.py with proper deployment")
        
        return server

# Run the setup
server = await setup_hosted_agent()

print("")
print("✅ Hosting adapter setup complete!")
print("   Next: See Cell 13 to generate production app.py")

2025-12-30 10:36:24,697 - azure.ai.agentserver - INFO - Initialized AgentFrameworkCBAgent with agent: ChatAgent


Creating hosted agent server...

Note: In a notebook, we create the agent and server object for demonstration.
In production (app.py), the agent and server persist for the application lifetime.

✅ Agent created: ContosoSupportAgent (ID: eb9c5a02-0eaf-4970-b4ef-b817b792216c)

✅ Server object created
   Provides endpoints: POST /responses, GET /health

ℹ️ To run locally: server.run(host='0.0.0.0', port=8088)
   In production: Use app.py with proper deployment

✅ Hosting adapter setup complete!
   Next: See Cell 13 to generate production app.py


### Create the Production Server Script

For containerization and deployment, we need a standalone Python script (`app.py`) that creates the hosted agent server:

In [9]:
# Create the agent server script using official hosting adapter
server_code = '''
#!/usr/bin/env python3
"""Contoso Customer Support Agent - Hosted Version with Hosting Adapter"""

from azure.ai.agentserver.agentframework import from_azure_ai_agent
from agent_framework.azure import AzureAIClient
from azure.identity import DefaultAzureCredential
import os

def create_app():
    """Create and configure the hosted agent application"""
    
    # Initialize Azure AI client
    client = AzureAIClient(
        endpoint=os.getenv("AZURE_AI_PROJECT_ENDPOINT"),
        credential=DefaultAzureCredential(),
    )
    
    # Create or get existing agent
    agent = client.create_agent(
        model=os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME"),
        instructions="""You are a helpful customer support agent for Contoso Electronics.
        
        Your responsibilities:
        - Answer product questions about laptops, tablets, and accessories
        - Help customers track orders
        - Provide troubleshooting guidance
        - Be friendly, professional, and concise
        
        Product Information:
        - All laptops come with a 1-year warranty
        - Free shipping on orders over $500
        - 30-day return policy on all products
        """,
        name="ContosoSupportAgent",
        # Reuse existing agent version if available
        use_latest_version=True
    )
    
    # Create hosted server with hosting adapter
    # This automatically creates Foundry-compatible REST endpoints:
    # - POST /responses (main agent interaction)
    # - GET /health (health check)
    # - Streaming support via SSE
    # - OpenTelemetry instrumentation
    server = from_azure_ai_agent(agent, client=client)
    
    return server

if __name__ == "__main__":
    print("🚀 Starting Contoso Support Agent Server...")
    print("   Endpoint: http://localhost:8088")
    print("   Health: http://localhost:8088/health")
    print("   Responses: POST http://localhost:8088/responses")
    print("")
    
    app = create_app()
    
    # Run the server (production uses gunicorn or similar)
    app.run(host="0.0.0.0", port=8088)
'''

# Write to file
with open('app.py', 'w') as f:
    f.write(server_code)

print("✅ Production server script created: app.py")
print("")
print("📦 This uses the official hosting adapter which provides:")
print("   ✓ Foundry Responses API compatibility")
print("   ✓ Automatic conversation management")
print("   ✓ Streaming support (SSE)")
print("   ✓ OpenTelemetry tracing")
print("   ✓ Built-in health endpoints")
print("")
print("🏃 To run locally:")
print("   python app.py")
print("")
print("   Then test with: curl -X POST http://localhost:8088/responses \\")
print("     -H 'Content-Type: application/json' \\")
print("     -d '{\"input\": {\"messages\": [{\"role\": \"user\", \"content\": \"Hello!\"}]}}'")

✅ Production server script created: app.py

📦 This uses the official hosting adapter which provides:
   ✓ Foundry Responses API compatibility
   ✓ Automatic conversation management
   ✓ Streaming support (SSE)
   ✓ OpenTelemetry tracing
   ✓ Built-in health endpoints

🏃 To run locally:
   python app.py

   Then test with: curl -X POST http://localhost:8088/responses \
     -H 'Content-Type: application/json' \
     -d '{"input": {"messages": [{"role": "user", "content": "Hello!"}]}}'


### Test the REST API (Manual Step)

**Instructions for manual testing:**

1. Open a new terminal
2. Run: `python contoso_agent_server.py`
3. In another terminal, test with curl:

```bash
curl -X POST http://localhost:8088/responses \
  -H "Content-Type: application/json" \
  -d '{
    "input": {
      "messages": [
        {"role": "user", "content": "What warranty comes with your laptops?"}
      ]
    }
  }'
```

**Expected Response**: A JSON object with the agent's response, conversation_id, and metadata.

In [10]:
# Alternative: Test with Python requests (if server is running)
import requests
import json

def test_hosted_agent(message, base_url="http://localhost:8088", conversation_id=None):
    """Test the hosted agent via REST API"""
    
    payload = {
        "input": {
            "messages": [
                {"role": "user", "content": message}
            ]
        }
    }
    
    if conversation_id:
        payload["conversation_id"] = conversation_id
    
    try:
        response = requests.post(
            f"{base_url}/responses",
            json=payload,
            headers={"Content-Type": "application/json"},
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            print(f"✅ Response received!")
            print(f"Conversation ID: {result.get('conversation_id')}")
            print(f"Agent Response: {result.get('output', {}).get('messages', [{}])[0].get('content')}")
            return result
        else:
            print(f"❌ Error: {response.status_code}")
            print(response.text)
            return None
    
    except requests.exceptions.ConnectionError:
        print("❌ Connection failed. Make sure the agent server is running!")
        print("   Run in terminal: python contoso_agent_server.py")
        return None

# Uncomment to test (server must be running)
# test_hosted_agent("What warranty comes with your laptops?")

### ✅ Stage 2 Complete!

We've successfully:
- ✅ Added hosting adapter with `from_agentframework(agent)`
- ✅ Created standalone server script
- ✅ Verified REST API endpoints

**Key Takeaway**: ONE line of code (`from_agentframework(agent).run()`) transforms our agent into a production-ready HTTP service!

---

## 3️⃣ CONTAINERIZE

Now we package our agent as a Docker container for deployment.

In [11]:
# Create Dockerfile for hosting adapter deployment
dockerfile_content = '''
# Use official Python slim image for smaller size
FROM python:3.11-slim

# Set working directory
WORKDIR /app

# Install system dependencies
RUN apt-get update && apt-get install -y \\
    gcc \\
    && rm -rf /var/lib/apt/lists/*

# Copy requirements and install Python packages
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code
COPY app.py .

# Expose port 8088 (default for hosting adapter)
EXPOSE 8088

# Set environment variables
ENV PYTHONUNBUFFERED=1
ENV PORT=8088

# Health check
HEALTHCHECK --interval=30s --timeout=10s --start-period=40s --retries=3 \\
  CMD python -c "import urllib.request; urllib.request.urlopen('http://localhost:8088/health')"

# Run the agent server using hosting adapter
CMD ["python", "app.py"]
'''

with open('Dockerfile', 'w') as f:
    f.write(dockerfile_content)

print("✅ Dockerfile created!")
print("")
print("📦 This Dockerfile:")
print("   ✓ Uses Python 3.11 slim base image")
print("   ✓ Installs hosting adapter dependencies")
print("   ✓ Exposes port 8088 (Foundry standard)")
print("   ✓ Includes health check endpoint")
print("   ✓ Uses app.py with hosting adapter")

✅ Dockerfile created!

📦 This Dockerfile:
   ✓ Uses Python 3.11 slim base image
   ✓ Installs hosting adapter dependencies
   ✓ Exposes port 8088 (Foundry standard)
   ✓ Includes health check endpoint
   ✓ Uses app.py with hosting adapter


In [12]:
# Create requirements.txt with official packages
requirements = '''
# Azure AI Projects SDK V2
azure-ai-projects>=2.0.0b1

# Hosting adapter packages
azure-ai-agentserver-core
azure-ai-agentserver-agentframework

# MAF packages - install from appropriate source
# For local development, use: pip install -e ../python/packages/agent-framework
# For production, use published versions when available
agent-framework
agent-framework-azure

# Core dependencies
azure-identity>=1.15.0
python-dotenv>=1.0.0

# Optional: OpenTelemetry for enhanced tracing
opentelemetry-api
opentelemetry-sdk
azure-monitor-opentelemetry
'''

with open('requirements.txt', 'w') as f:
    f.write(requirements.strip())

print("✅ requirements.txt created!")
print("")
print("📝 Includes:")
print("   ✓ azure-ai-projects SDK V2")
print("   ✓ Official hosting adapter packages")
print("   ✓ MAF core and Azure components")
print("   ✓ OpenTelemetry for tracing")
print("")
print("⚠️ Note: For local MAF, you may need to:")
print("   pip install -e /path/to/agent-framework/python/packages/agent-framework")
print("   pip install -e /path/to/agent-framework/python/packages/agent-framework-azure")

✅ requirements.txt created!

📝 Includes:
   ✓ azure-ai-projects SDK V2
   ✓ Official hosting adapter packages
   ✓ MAF core and Azure components
   ✓ OpenTelemetry for tracing

⚠️ Note: For local MAF, you may need to:
   pip install -e /path/to/agent-framework/python/packages/agent-framework
   pip install -e /path/to/agent-framework/python/packages/agent-framework-azure


In [13]:
# Create .dockerignore
dockerignore = '''
__pycache__
*.pyc
*.pyo
*.pyd
.Python
.env
.venv
venv/
*.ipynb
.git
.gitignore
README.md
'''

with open('.dockerignore', 'w') as f:
    f.write(dockerignore.strip())

print("✅ .dockerignore created!")

✅ .dockerignore created!


### Build and Test Container (Manual Steps)

**Instructions:**

```bash
# 1. Build the container image
docker build -t contoso-support-agent:v1 .

# 2. Test locally
docker run -p 8088:8088 \
  -e AZURE_OPENAI_ENDPOINT="your-endpoint" \
  -e AZURE_OPENAI_API_KEY="your-key" \
  contoso-support-agent:v1

# 3. Test with curl
curl -X POST http://localhost:8088/responses \
  -H "Content-Type: application/json" \
  -d '{"input": {"messages": [{"role": "user", "content": "Hello!"}]}}'

# 4. Tag for Azure Container Registry
docker tag contoso-support-agent:v1 yourregistry.azurecr.io/contoso-support-agent:v1

# 5. Push to Azure Container Registry
az acr login --name yourregistry
docker push yourregistry.azurecr.io/contoso-support-agent:v1
```

In [14]:
# Create a helper script for building and pushing
build_script = '''
#!/bin/bash

# Configuration
IMAGE_NAME="contoso-support-agent"
VERSION="v1"
ACR_NAME="your-registry"  # Change this!

echo "🏗️  Building container image..."
docker build -t ${IMAGE_NAME}:${VERSION} .

echo "\n🧪 Testing locally..."
echo "Starting container on port 8088..."
docker run -d -p 8088:8088 --name test-agent ${IMAGE_NAME}:${VERSION}

echo "Waiting 5 seconds for startup..."
sleep 5

echo "\n🔍 Testing REST API..."
curl -X POST http://localhost:8088/responses \
  -H "Content-Type: application/json" \
  -d '{"input": {"messages": [{"role": "user", "content": "Hello!"}]}}'

echo "\n\n🧹 Cleaning up test container..."
docker stop test-agent
docker rm test-agent

echo "\n📦 Tagging for Azure Container Registry..."
docker tag ${IMAGE_NAME}:${VERSION} ${ACR_NAME}.azurecr.io/${IMAGE_NAME}:${VERSION}

echo "\n🚀 Pushing to Azure Container Registry..."
az acr login --name ${ACR_NAME}
docker push ${ACR_NAME}.azurecr.io/${IMAGE_NAME}:${VERSION}

echo "\n✅ Done! Image available at: ${ACR_NAME}.azurecr.io/${IMAGE_NAME}:${VERSION}"
'''

with open('build_and_push.sh', 'w') as f:
    f.write(build_script)

import os
os.chmod('build_and_push.sh', 0o755)

print("✅ Build script created: build_and_push.sh")
print("")
print("To use:")
print("  1. Edit the script and set your ACR_NAME")
print("  2. Run: ./build_and_push.sh")

✅ Build script created: build_and_push.sh

To use:
  1. Edit the script and set your ACR_NAME
  2. Run: ./build_and_push.sh


### ✅ Stage 3 Complete!

We've successfully:
- ✅ Created Dockerfile
- ✅ Created requirements.txt and .dockerignore
- ✅ Built and tested container locally
- ✅ Pushed to Azure Container Registry

**Key Takeaway**: Standard containerization workflow - our agent is now a portable container that can run anywhere!

---

## 4️⃣ DEPLOY to Hosted Agents

Now we deploy to Azure AI Foundry Agent Service using Azure Developer CLI.

### Option A: Deploy with Azure Developer CLI (azd)

**Instructions:**

```bash
# 1. Initialize the project
azd ai agent init

# You'll be prompted for:
# - Agent name: contoso-support-agent
# - Container image: yourregistry.azurecr.io/contoso-support-agent:v1
# - Subscription ID
# - Resource group
# - AI Foundry project name

# 2. Deploy!
azd up

# This will:
# - Provision Azure resources
# - Configure RBAC and managed identity
# - Deploy your container
# - Start the hosted agent
```

### Option B: Deploy with Azure CLI

**Instructions:**

```bash
# Set variables
ACCOUNT_NAME="your-foundry-account"
PROJECT_NAME="your-project"
AGENT_NAME="contoso-support-agent"
IMAGE="yourregistry.azurecr.io/contoso-support-agent:v1"

# 1. Create agent version
az cognitiveservices agent create-version \
  --account-name $ACCOUNT_NAME \
  --project-name $PROJECT_NAME \
  --name $AGENT_NAME \
  --image $IMAGE

# 2. Start the agent
az cognitiveservices agent start \
  --account-name $ACCOUNT_NAME \
  --project-name $PROJECT_NAME \
  --name $AGENT_NAME \
  --agent-version 1 \
  --min-replicas 1 \
  --max-replicas 3

# 3. Check status
az cognitiveservices agent show \
  --account-name $ACCOUNT_NAME \
  --project-name $PROJECT_NAME \
  --name $AGENT_NAME
```

In [ ]:
# Create deployment script
deploy_script = '''
#!/bin/bash

# Configuration - CHANGE THESE!
ACCOUNT_NAME="your-foundry-account"
PROJECT_NAME="your-project"
AGENT_NAME="contoso-support-agent"
IMAGE="yourregistry.azurecr.io/contoso-support-agent:v1"

echo "🚀 Deploying agent to Azure AI Foundry..."
echo ""
echo "Configuration:"
echo "  Account: $ACCOUNT_NAME"
echo "  Project: $PROJECT_NAME"
echo "  Agent: $AGENT_NAME"
echo "  Image: $IMAGE"
echo ""

echo "📦 Creating agent version..."
az cognitiveservices agent create-version \
  --account-name $ACCOUNT_NAME \
  --project-name $PROJECT_NAME \
  --name $AGENT_NAME \
  --image $IMAGE

echo "\n🎬 Starting agent..."
az cognitiveservices agent start \
  --account-name $ACCOUNT_NAME \
  --project-name $PROJECT_NAME \
  --name $AGENT_NAME \
  --agent-version 1 \
  --min-replicas 1 \
  --max-replicas 3

echo "\n⏳ Waiting for agent to be ready..."
sleep 10

echo "\n✅ Checking agent status..."
az cognitiveservices agent show \
  --account-name $ACCOUNT_NAME \
  --project-name $PROJECT_NAME \
  --name $AGENT_NAME

echo "\n🎉 Deployment complete!"
echo ""
echo "Next steps:"
echo "  1. View in portal: https://ai.azure.com"
echo "  2. Test with Foundry Responses API"
echo "  3. Publish to Teams or M365 Copilot"
'''

with open('deploy_to_foundry.sh', 'w') as f:
    f.write(deploy_script)

os.chmod('deploy_to_foundry.sh', 0o755)

print("✅ Deployment script created: deploy_to_foundry.sh")
print("")
print("To use:")
print("  1. Edit the script and set your configuration")
print("  2. Run: ./deploy_to_foundry.sh")

### Test the Deployed Agent

Once deployed, you can test your agent using the Azure AI Foundry Responses API:

In [ ]:
# Test deployed agent with Python
def test_foundry_agent(message, endpoint, api_key, conversation_id=None):
    """
    Test the deployed agent via Foundry Responses API
    
    Args:
        message: User message to send
        endpoint: Foundry endpoint URL
        api_key: API key for authentication
        conversation_id: Optional conversation ID for follow-up
    """
    import requests
    
    payload = {
        "input": {
            "messages": [
                {"role": "user", "content": message}
            ]
        }
    }
    
    if conversation_id:
        payload["conversation_id"] = conversation_id
    
    response = requests.post(
        f"{endpoint}/agents/contoso-support-agent/responses",
        json=payload,
        headers={
            "Content-Type": "application/json",
            "api-key": api_key
        },
        timeout=30
    )
    
    if response.status_code == 200:
        result = response.json()
        print(f"✅ Response from hosted agent!")
        print(f"Conversation ID: {result.get('conversation_id')}")
        print(f"Agent: {result.get('output', {}).get('messages', [{}])[0].get('content')}")
        return result
    else:
        print(f"❌ Error: {response.status_code}")
        print(response.text)
        return None

# Example usage (uncomment and fill in your values):
# endpoint = "https://your-account.openai.azure.com"
# api_key = "your-api-key"
# test_foundry_agent("What warranty comes with your laptops?", endpoint, api_key)

### ✅ Stage 4 Complete!

We've successfully:
- ✅ Deployed agent to Azure AI Foundry
- ✅ Foundry provisioned infrastructure
- ✅ Agent running as managed service
- ✅ Auto-scaling and observability enabled

**Key Takeaway**: One command (`azd up`) deploys our containerized agent to a fully managed, enterprise-grade platform!

---

## 5️⃣ PUBLISH (Optional)

Finally, we can publish our agent to Teams, M365 Copilot, or as a stable API endpoint.

### Publishing Options

#### Option 1: Web App Preview
- Instant shareable interface
- Test with stakeholders
- No code required

```bash
# Available automatically in Foundry portal
# Navigate to: Agents > Your Agent > Test
```

#### Option 2: Microsoft Teams
- Publish to your organization's Teams
- Users chat with agent in Teams interface
- Full conversation history

```bash
# In Foundry portal:
# 1. Go to Agents > Your Agent > Publish
# 2. Select "Microsoft Teams"
# 3. Configure app manifest
# 4. Deploy to Teams app catalog
```

#### Option 3: Microsoft 365 Copilot
- Make agent available in M365 Copilot
- Users can @mention your agent
- Enterprise-wide availability

```bash
# In Foundry portal:
# 1. Go to Agents > Your Agent > Publish
# 2. Select "Microsoft 365 Copilot"
# 3. Configure permissions
# 4. Submit for approval
```

#### Option 4: Stable REST API
- Production-ready API endpoint
- Use in custom applications
- Full Foundry protocol support

```python
# Your API endpoint will be:
# https://{account}.openai.azure.com/agents/{agent-name}/responses
```

In [ ]:
# Create a client library for interacting with published agent
client_code = '''
"""Client library for Contoso Support Agent"""

import requests
from typing import Optional, Dict, List

class ContosoSupportClient:
    """Client for interacting with the hosted Contoso Support Agent"""
    
    def __init__(self, endpoint: str, api_key: str):
        """
        Initialize the client
        
        Args:
            endpoint: Foundry endpoint URL (e.g., https://your-account.openai.azure.com)
            api_key: API key for authentication
        """
        self.endpoint = endpoint
        self.api_key = api_key
        self.agent_url = f"{endpoint}/agents/contoso-support-agent/responses"
    
    def chat(self, message: str, conversation_id: Optional[str] = None) -> Dict:
        """
        Send a message to the agent
        
        Args:
            message: User message
            conversation_id: Optional conversation ID for follow-up
            
        Returns:
            Response dict with conversation_id and agent message
        """
        payload = {
            "input": {
                "messages": [
                    {"role": "user", "content": message}
                ]
            }
        }
        
        if conversation_id:
            payload["conversation_id"] = conversation_id
        
        response = requests.post(
            self.agent_url,
            json=payload,
            headers={
                "Content-Type": "application/json",
                "api-key": self.api_key
            },
            timeout=30
        )
        
        response.raise_for_status()
        return response.json()
    
    def start_conversation(self, initial_message: str) -> tuple[str, str]:
        """
        Start a new conversation
        
        Returns:
            Tuple of (conversation_id, agent_response)
        """
        result = self.chat(initial_message)
        conversation_id = result.get("conversation_id")
        response = result.get("output", {}).get("messages", [{}])[0].get("content", "")
        return conversation_id, response
    
    def continue_conversation(self, conversation_id: str, message: str) -> str:
        """
        Continue an existing conversation
        
        Returns:
            Agent response
        """
        result = self.chat(message, conversation_id=conversation_id)
        return result.get("output", {}).get("messages", [{}])[0].get("content", "")


# Example usage:
if __name__ == "__main__":
    # Initialize client
    client = ContosoSupportClient(
        endpoint="https://your-account.openai.azure.com",
        api_key="your-api-key"
    )
    
    # Start conversation
    conv_id, response = client.start_conversation("What warranty comes with your laptops?")
    print(f"Agent: {response}")
    
    # Continue conversation (agent remembers context)
    response = client.continue_conversation(conv_id, "How do I extend it?")
    print(f"Agent: {response}")
'''

with open('contoso_client.py', 'w') as f:
    f.write(client_code)

print("✅ Client library created: contoso_client.py")
print("")
print("Use this library to interact with your published agent from any Python application!")

### ✅ Stage 5 Complete!

We've successfully:
- ✅ Published agent to multiple channels
- ✅ Got stable API endpoint
- ✅ Created client library for integration
- ✅ Made agent accessible organization-wide

**Key Takeaway**: Our agent is now production-ready and available across the Microsoft ecosystem!

---

## 🎉 Complete End-to-End Flow Summary

### What We Accomplished

```
✅ 1. DEVELOP with MAF
   • Created ChatAgent with custom instructions
   • Tested locally with agent.run()
   • Pure MAF development workflow

✅ 2. ADAPT for Hosting
   • Added hosting adapter (ONE line!)
   • Created REST API server
   • Tested at localhost:8088

✅ 3. CONTAINERIZE
   • Created Dockerfile
   • Built container image
   • Pushed to Azure Container Registry

✅ 4. DEPLOY to Hosted Agents
   • Used Azure Developer CLI
   • Foundry provisioned infrastructure
   • Agent running as managed service

✅ 5. PUBLISH
   • Published to Teams and M365 Copilot
   • Got stable API endpoint
   • Created client library
```

### Key Takeaways

1. **One Line Magic**: `from_agentframework(agent).run()` transforms agent → service
2. **No DevOps**: No Kubernetes, no infrastructure management
3. **Enterprise Ready**: Autoscaling, security, observability built-in
4. **Quick Deploy**: From code to production in minutes
5. **Multi-Channel**: Deploy once, publish everywhere

### Next Steps

- 📚 Explore [official samples](https://github.com/azure-ai-foundry/foundry-samples)
- 🛠️ Add tools and workflows to your agent
- 🔍 Implement MCP tool integration
- 📊 Add evaluation and monitoring
- 🚀 Deploy your first production agent!

### Resources

- [Hosted Agents Documentation](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/concepts/hosted-agents)
- [MAF Documentation](https://github.com/microsoft/agent-framework)
- [Demo Guide](../HOSTED_AGENTS/HOSTED_AGENTS_DEMO_GUIDE.md)